# 메모리 추가하여 동작 점검

In [13]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.
            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

In [15]:
classify_message_chain = message_type_prompt | model 
classify_message_chain

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.\n            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'\n            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000212C1A8E170>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000212C1A8CA30>, openai

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [17]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_message_with_memory = RunnableWithMessageHistory(
    classify_message_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [18]:
from langchain_core.runnables import RunnablePassthrough

classify_message_with_memory_chain = RunnablePassthrough.assign(msg_type=classify_message_with_memory)

In [19]:
classify_message_with_memory_chain.invoke(
    {"input": "주문 상태가 '주문 완료'인 주문을 알려줘", 
     "user_id": 1},
    config={"configurable": {"session_id": "test_240514-1"}}
    )

Parent run 0b234c89-2aa5-4b37-81b1-4b652bd0cf01 not found for run 0af611e4-db35-48ec-b8e4-c9d2621da63c. Treating as a root run.


{'input': "주문 상태가 '주문 완료'인 주문을 알려줘",
 'user_id': 1,
 'msg_type': AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 138, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9f615600-c11f-404b-879a-98d1fce2587f-0')}

In [42]:
def inquiry_request_route(info):
    print("inquiry_request_route 입력 데이터 -> ", info)
    if "문의" in info["msg_type"].content.lower():
        return handle_inquiry_chain
    else:
        return "요청 문의 처리 체인 구현 예정"

In [43]:
from langchain_core.prompts import ChatPromptTemplate

classify_inquiry_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.
            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.
            -주문 상품에 관한 문의: '주문 완료'
            -주문 변경에 관한 문의: '주문 변경'
            -주문 취소에 관한 문의: '주문 취소'
            """
        ),
        ("human", "{input}"),

    ]
)

In [44]:
classify_inquiry_chain = RunnablePassthrough.assign(inquiry_type=classify_inquiry_prompt | model)

In [45]:
def get_completed_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의 주문 완료된 주문내역 조회 완료"

def get_payment_completed_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의입금 완료된 주문내역 조회 완료"

def get_changed_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의주문 변경된 주문내역 조회 완료"

def get_canceled_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의주문 취소된 주문내역 조회 완료"

In [59]:
from langchain_core.runnables import RunnableLambda

def inquiry_types_route(info):
    print("inquiry_types_route 함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["inquiry_type"].content.lower():
        return RunnableLambda(get_changed_orders)
    elif "주문 취소" in info["inquiry_type"].content.lower():
        return RunnableLambda(get_canceled_orders)
    else:
        return RunnableLambda(get_completed_orders)

In [60]:
from langchain_core.runnables import RunnableLambda

handle_inquiry_chain = classify_inquiry_chain | RunnableLambda(inquiry_types_route)

In [61]:
full_chain = classify_message_with_memory_chain | inquiry_request_route
full_chain

RunnableAssign(mapper={
  msg_type: RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
            }), config={'run_name': 'insert_history'})
            | RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.\n            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'\n            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'\n            ")), Mes

In [62]:

full_chain.invoke(
    {"input": "주문 변경 내역을 알고 싶어", 
     "user_id": 1},
     config={"configurable": {"session_id": "test_240514-1"}}
    )

Parent run da5a59d5-1f05-4c11-ae45-53c6df4c3369 not found for run 24f46938-3eed-4e9d-a2c6-9d955b5632b3. Treating as a root run.


inquiry_request_route 입력 데이터 ->  {'input': '주문 변경 내역을 알고 싶어', 'user_id': 1, 'msg_type': AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 129, 'total_tokens': 131}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5fb93a43-ebec-4850-95aa-fb1afc5fe04e-0')}
inquiry_types_route 함수로 전달된 데이터 ->  {'input': '주문 변경 내역을 알고 싶어', 'user_id': 1, 'msg_type': AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 129, 'total_tokens': 131}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5fb93a43-ebec-4850-95aa-fb1afc5fe04e-0'), 'inquiry_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 146, 'total_tokens': 149}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6f66

'사용자 1의주문 변경된 주문내역 조회 완료'

In [58]:
store["test_240514-1"]
del store["test_240514-1"]